In [ ]:
import keras
from keras.layers import Dense, Dropout, Input, MaxPooling2D, Conv2D, Flatten, MaxPool2D, AveragePooling2D, GlobalAveragePooling2D, BatchNormalization
from keras.models import Sequential, Model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
from keras.applications.densenet import DenseNet121


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/cassava/train_tfrecords"

In [ ]:
%cd drive/MyDrive/cassava/train_tfrecords/

/content/drive/MyDrive/cassava/train_tfrecords


In [ ]:
from data_loader import initialize_dataset

In [ ]:
# Get a list of the training record files
# tfrecord_files = glob.glob(os.path.join("data", "train_tfrecords", "*.tfrec"))
tfrecord_files = glob.glob(os.path.join(path, "*.tfrec"))


# Define the number of files for training, 70% of files
n_file_train = int(0.85 * len(tfrecord_files))

# Split the record files into 70% training and 30% validation
tfrecord_files_train = tfrecord_files[:n_file_train]
tfrecord_files_valid = tfrecord_files[n_file_train:]

In [ ]:
BATCH_SIZE = 32

IMG_SIZE = (256, 256)

# Initialize the training data loader, with random flipping and rotations
dataset_train = initialize_dataset(tfrecord_files_train, batch_size=BATCH_SIZE, img_size=IMG_SIZE, flip=True, rot=(-1, 1))

# Initialize the validation loader, with no flipping or rotations
dataset_valid = initialize_dataset(tfrecord_files_valid, img_size=IMG_SIZE)

In [ ]:
def save_model(model):
  import os
  model_name = 'checkpoint1.h5'
  save_dir = os.path.join(os.getcwd(), 'saved_models')
  
  # Save model and weights
  if not os.path.isdir(save_dir):
      os.makedirs(save_dir)
  model_path = os.path.join(save_dir, model_name)
  model.save(model_path, overwrite=True)
  print('Saved trained model at %s ' % model_path)

In [ ]:
# Our most basic DenseNet model, just a global average pooling and a fully connected output layer following DenseNet121. No layers frozen.
# Performs suprisingly well, with val loss of 0.47 and val accuracy of 0.84

initial_lr = 5e-5
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_lr,
    decay_steps=10000,
    decay_rate=0.9)

opt = keras.optimizers.Adam(learning_rate=lr_schedule)

early_stop = EarlyStopping(monitor='val_loss', mode='min', 
                       patience=5, restore_best_weights=True, verbose=1)

def build_model():
  model_dn = DenseNet121(
      include_top= False,
      weights="imagenet",
      input_shape= (*IMG_SIZE, 3),
  )

  x = GlobalAveragePooling2D()(model_dn.output)
  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)
  output = Dense(5, activation='softmax')(x)

  model = Model(inputs = model_dn.input, outputs = output)

  return model

In [ ]:
model = build_model()

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer = opt,
    metrics=["accuracy"]
)

# model.summary()

In [ ]:
train_hist = model.fit(
    x = dataset_train,
    epochs = 100,
    validation_data = dataset_valid,
    callbacks = [early_stop]
)

Epoch 1/100
544/544 [==============================] - 169s 292ms/step - loss: 1.1666 - accuracy: 0.5905 - val_loss: 0.6562 - val_accuracy: 0.7887
Epoch 2/100
544/544 [==============================] - 157s 288ms/step - loss: 0.6046 - accuracy: 0.8037 - val_loss: 0.5860 - val_accuracy: 0.8154
Epoch 3/100
544/544 [==============================] - 157s 289ms/step - loss: 0.5130 - accuracy: 0.8316 - val_loss: 0.5509 - val_accuracy: 0.8226
Epoch 4/100
544/544 [==============================] - 157s 288ms/step - loss: 0.4424 - accuracy: 0.8517 - val_loss: 0.4889 - val_accuracy: 0.8416
Epoch 5/100
544/544 [==============================] - 157s 288ms/step - loss: 0.4145 - accuracy: 0.8609 - val_loss: 0.4907 - val_accuracy: 0.8456
Epoch 6/100
544/544 [==============================] - 157s 289ms/step - loss: 0.3736 - accuracy: 0.8725 - val_loss: 0.5081 - val_accuracy: 0.8409
Epoch 7/100
544/544 [==============================] - 157s 288ms/step - loss: 0.3450 - accuracy: 0.8805 - val_loss: 0

In [ ]:
# Adding more complexity to our model above, with two fully-connected hidden layers before the output
# More dropout with slightly higher dropout rate to mitigate overfitting
# Val loss as low as 0.43, and val accuracy pushes up to 0.85

initial_lr = 5e-5
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_lr,
    decay_steps=10000,
    decay_rate=0.9)

# reduce_lr = ReduceLROnPlateau(
#     monitor="val_loss",
#     factor=0.5,
#     patience=5,
#     verbose=1,
#     min_lr=1e-6
# )

opt = keras.optimizers.Adam(learning_rate=lr_schedule)

early_stop = EarlyStopping(monitor='val_loss', mode='min', 
                       patience=5, restore_best_weights=True, verbose=1)

def build_model2():
  model_dn = DenseNet121(
      include_top= False,
      weights="imagenet",
      input_shape= (*IMG_SIZE, 3),
  )

  x = GlobalAveragePooling2D()(model_dn.output)
  x = BatchNormalization()(x)
  x = Dropout(0.4)(x)
  x = Dense(1024,activation='relu', kernel_initializer="he_normal")(x) 
  x = Dropout(0.4)(x)
  x = Dense(512,activation='relu', kernel_initializer="he_normal")(x) 
  x = BatchNormalization()(x)
  x = Dropout(0.4)(x)
  output = Dense(5, activation='softmax')(x)

  model = Model(inputs = model_dn.input, outputs = output)

  return model

In [ ]:
model2 = build_model2()

model2.compile(
    loss="sparse_categorical_crossentropy",
    optimizer = opt,
    metrics=["accuracy"]
)

# model2.summary()

In [ ]:
train_hist2 = model2.fit(
    x = dataset_train,
    epochs = 100,
    validation_data = dataset_valid,
    callbacks = [early_stop]
)

Epoch 1/100
544/544 [==============================] - 171s 292ms/step - loss: 1.9385 - accuracy: 0.3818 - val_loss: 1.0109 - val_accuracy: 0.6752
Epoch 2/100
544/544 [==============================] - 162s 297ms/step - loss: 1.0681 - accuracy: 0.6785 - val_loss: 0.6549 - val_accuracy: 0.7887
Epoch 3/100
544/544 [==============================] - 160s 294ms/step - loss: 0.8582 - accuracy: 0.7452 - val_loss: 0.6301 - val_accuracy: 0.7982
Epoch 4/100
544/544 [==============================] - 159s 291ms/step - loss: 0.7345 - accuracy: 0.7810 - val_loss: 0.5529 - val_accuracy: 0.8234
Epoch 5/100
544/544 [==============================] - 159s 291ms/step - loss: 0.6679 - accuracy: 0.7967 - val_loss: 0.5937 - val_accuracy: 0.8179
Epoch 6/100
544/544 [==============================] - 163s 299ms/step - loss: 0.6337 - accuracy: 0.8032 - val_loss: 0.5160 - val_accuracy: 0.8409
Epoch 7/100
544/544 [==============================] - 164s 301ms/step - loss: 0.5933 - accuracy: 0.8146 - val_loss: 0

In [ ]:
# see how transfer learning does if we freeze base (DenseNet) layers

model_freeze_base = build_model2()

# Freeze all DenseNet layers
for layer in model_freeze_base.layers[:-9]:
  layer.trainable = False

initial_lr_transfer = 0.01
lr_schedule_transfer = keras.optimizers.schedules.ExponentialDecay(
    initial_lr_transfer,
    decay_steps=10000,
    decay_rate=0.9)

opt_transfer = keras.optimizers.Adam(learning_rate=lr_schedule_transfer)


model_freeze_base.compile(
    loss="sparse_categorical_crossentropy",
    optimizer = opt_transfer,
    metrics=["accuracy"]
)

# model_freeze_base.summary()

In [ ]:
# model clearly underfits

train_freeze_base = model_freeze_base.fit(
    x = dataset_train,
    epochs = 100,
    validation_data = dataset_valid,
    callbacks = [early_stop]
)

Epoch 1/100
544/544 [==============================] - 154s 272ms/step - loss: 1.4834 - accuracy: 0.5698 - val_loss: 1.0994 - val_accuracy: 0.6253
Epoch 2/100
544/544 [==============================] - 147s 269ms/step - loss: 1.0186 - accuracy: 0.6336 - val_loss: 1.3190 - val_accuracy: 0.6585
Epoch 3/100
544/544 [==============================] - 147s 269ms/step - loss: 0.9634 - accuracy: 0.6505 - val_loss: 0.9312 - val_accuracy: 0.6643
Epoch 4/100
544/544 [==============================] - 147s 269ms/step - loss: 0.9376 - accuracy: 0.6571 - val_loss: 0.9380 - val_accuracy: 0.6505
Epoch 5/100
544/544 [==============================] - 147s 270ms/step - loss: 0.9364 - accuracy: 0.6569 - val_loss: 0.9114 - val_accuracy: 0.6653
Epoch 6/100
544/544 [==============================] - 147s 270ms/step - loss: 0.9268 - accuracy: 0.6562 - val_loss: 0.8895 - val_accuracy: 0.6730
Epoch 7/100
544/544 [==============================] - 147s 271ms/step - loss: 0.9220 - accuracy: 0.6584 - val_loss: 0

In [ ]:
# try training model with partial freezing of DenseNet layers

model_freeze_partial = build_model2()

for layer in model_freeze_partial.layers[:12]:
  layer.trainable = False

model_freeze_partial.compile(
    loss="sparse_categorical_crossentropy",
    optimizer = opt,
    metrics=["accuracy"]
)

In [ ]:
# performance not far off from unfrozen model
# minimum val loss slightly higher at 0.45, val accuracy around the same 

train_freeze_partial = model_freeze_partial.fit(
    x = dataset_train,
    epochs = 100,
    validation_data = dataset_valid,
    callbacks = [early_stop]
)

Epoch 1/100
544/544 [==============================] - 204s 353ms/step - loss: 1.9706 - accuracy: 0.3888 - val_loss: 0.7639 - val_accuracy: 0.7482
Epoch 2/100
544/544 [==============================] - 186s 341ms/step - loss: 1.0917 - accuracy: 0.6730 - val_loss: 0.7034 - val_accuracy: 0.7949
Epoch 3/100
544/544 [==============================] - 190s 348ms/step - loss: 0.8715 - accuracy: 0.7416 - val_loss: 0.6011 - val_accuracy: 0.8259
Epoch 4/100
544/544 [==============================] - 184s 337ms/step - loss: 0.7680 - accuracy: 0.7788 - val_loss: 0.6109 - val_accuracy: 0.8189
Epoch 5/100
544/544 [==============================] - 184s 338ms/step - loss: 0.6925 - accuracy: 0.7880 - val_loss: 0.5431 - val_accuracy: 0.8386
Epoch 6/100
544/544 [==============================] - 184s 338ms/step - loss: 0.6329 - accuracy: 0.8034 - val_loss: 0.5218 - val_accuracy: 0.8476
Epoch 7/100
544/544 [==============================] - 184s 337ms/step - loss: 0.6067 - accuracy: 0.8125 - val_loss: 0

In [ ]:
save_model(model_freeze_partial)

Saved trained model at /content/drive/My Drive/cassava/train_tfrecords/saved_models/checkpoint1.h5 


In [ ]:
checkpoint = keras.models.load_model("saved_models/checkpoint1.h5")

In [ ]:
# Unfreeze previously frozen layers
for layer in checkpoint.layers:
  layer.trainable = True

# But freeze batch norm layers
for layer in checkpoint.layers:
  if "bn" in layer.name or "batch" in layer.name:
    layer.trainable = False

In [ ]:
# use smaller learning rate

initial_lr2 = 1e-5
lr_schedule2 = keras.optimizers.schedules.ExponentialDecay(
    initial_lr2,
    decay_steps=10000,
    decay_rate=0.9)

opt_fine = keras.optimizers.Adam(learning_rate=lr_schedule2)

checkpoint.compile(
    loss="sparse_categorical_crossentropy",
    optimizer = opt_fine,
    metrics=["accuracy"]
)

In [ ]:
# Val accuracy pushed up to 0.86, but val loss does not outperform original unfrozen model
# Given the imbalanced data set, this 1 percent increase in val accuracy with no decrease at all in val loss might not be significant

train_freeze_unfreeze2 = checkpoint.fit(
    x = dataset_train,
    epochs = 100,
    validation_data = dataset_valid,
    callbacks = [early_stop]
)

Epoch 1/100
544/544 [==============================] - 216s 385ms/step - loss: 0.4190 - accuracy: 0.8624 - val_loss: 0.4418 - val_accuracy: 0.8549
Epoch 2/100
544/544 [==============================] - 209s 383ms/step - loss: 0.3984 - accuracy: 0.8648 - val_loss: 0.4439 - val_accuracy: 0.8601
Epoch 3/100
544/544 [==============================] - 208s 382ms/step - loss: 0.3922 - accuracy: 0.8657 - val_loss: 0.4774 - val_accuracy: 0.8481
Epoch 4/100
544/544 [==============================] - 208s 382ms/step - loss: 0.3832 - accuracy: 0.8735 - val_loss: 0.4388 - val_accuracy: 0.8664
Epoch 5/100
544/544 [==============================] - 208s 382ms/step - loss: 0.3839 - accuracy: 0.8688 - val_loss: 0.4365 - val_accuracy: 0.8621
Epoch 6/100
544/544 [==============================] - 207s 380ms/step - loss: 0.3736 - accuracy: 0.8728 - val_loss: 0.4339 - val_accuracy: 0.8651
Epoch 7/100
544/544 [==============================] - 207s 380ms/step - loss: 0.3651 - accuracy: 0.8752 - val_loss: 0

In [ ]:
# Try training model with some partial freezing, trial 2
# Further trials not shown in notebook to avoid clutter, but as we increase the number of frozen blocks in the base DenseNet model,
# validation performance starts to noticeably go down 


model_freeze_partial2 = build_model2()

for layer in model_freeze_partial2.layers[:33]:
  layer.trainable = False

model_freeze_partial2.compile(
    loss="sparse_categorical_crossentropy",
    optimizer = opt,
    metrics=["accuracy"]
)

In [ ]:
train_freeze_partial2 = model_freeze_partial2.fit(
    x = dataset_train,
    epochs = 100,
    validation_data = dataset_valid,
    callbacks = [early_stop]
)

Epoch 1/100
544/544 [==============================] - 192s 336ms/step - loss: 1.6776 - accuracy: 0.4844 - val_loss: 0.7807 - val_accuracy: 0.7664
Epoch 2/100
544/544 [==============================] - 181s 332ms/step - loss: 0.9409 - accuracy: 0.7129 - val_loss: 0.6230 - val_accuracy: 0.8074
Epoch 3/100
544/544 [==============================] - 182s 334ms/step - loss: 0.8079 - accuracy: 0.7482 - val_loss: 0.5820 - val_accuracy: 0.8184
Epoch 4/100
544/544 [==============================] - 181s 332ms/step - loss: 0.7126 - accuracy: 0.7776 - val_loss: 0.5830 - val_accuracy: 0.8224
Epoch 5/100
544/544 [==============================] - 181s 332ms/step - loss: 0.6817 - accuracy: 0.7848 - val_loss: 0.5501 - val_accuracy: 0.8241
Epoch 6/100
544/544 [==============================] - 181s 333ms/step - loss: 0.6151 - accuracy: 0.7986 - val_loss: 0.5164 - val_accuracy: 0.8304
Epoch 7/100
544/544 [==============================] - 181s 332ms/step - loss: 0.5890 - accuracy: 0.8128 - val_loss: 0